In [ ]:
from src.sshDb import ssh_db
import pandas as pd
import xlrd  # Needed to read the Excel file

# Read excel file to dataframe.
# File must be inside 'files-to-import' folder with the name 'Elektrik Üretim Lisanslar_.xls'
epdk_list = pd.read_excel('../files4update/Elektrik Üretim Lisanslar_.xls', header=[0, 1])

# Filter only important rows to reduce memory for later.
epdk_list = epdk_list.loc[
    (epdk_list['Lisans Durumu', 'Unnamed: 3_level_1'] == 'Sonlandırıldı') |
    (epdk_list['Lisans Durumu', 'Unnamed: 3_level_1'] == 'Yürürlükte') |
    (epdk_list['Lisans Durumu', 'Unnamed: 3_level_1'] == 'İptal Edildi')
    ]

# Split multiindex columns for convenience
epdk_list_l0 = epdk_list.droplevel(1, axis=1)
epdk_list_l1 = epdk_list.droplevel(0, axis=1)

# Filter only important rows and remove multiindex, again for the same reasons
epdk_list_new = epdk_list.loc[epdk_list['Lisans Durumu', 'Unnamed: 3_level_1'] == 'Yürürlükte']
epdk_list_new = epdk_list_new.droplevel(1, axis=1)
# For some reason a list doesn't work
epdk_list_new = epdk_list_new[['Lisans No', 'Tesis Adı']]

# Start SSH tunnel and connect to DB
ssh_db.connect_ssh_db()
# Get table wf from database
wfPostgre = pd.read_sql_table('wf', ssh_db.engine)
current_version = wfPostgre['version'].max()
wfPostgre = wfPostgre.loc[wfPostgre['version'] == current_version]


#####

# Identify rows not in the database
df_to_add = epdk_list
df_to_add = df_to_add.droplevel(1, axis=1)
df_to_add = df_to_add[['Lisans No', 'Tesis Adı']]

df_to_add = df_to_add[~df_to_add["Lisans No"].isin(wfPostgre["Lisans No"])] 

# Concatenate new rows with the database table
wfPostgre = pd.concat([df_to_add, wfPostgre], axis=0, ignore_index=True, sort=False)

#####

# '~' means not in
df1_filtered = epdk_list_new[~epdk_list_new['Lisans No'].isin(wfPostgre['Lisans No'])]

wfPostgre = pd.concat([df1_filtered, wfPostgre], axis=0, ignore_index=True, sort=False)


# UPDATE EXISTING ROWS IN TABLE WF
# set index columns to match different tables
wfPostgre = wfPostgre.set_index('Lisans No')
epdk_list_l0 = epdk_list_l0.set_index('Lisans No')
epdk_list_l1 = epdk_list_l1.set_index('Unnamed: 6_level_1')


# update the existing licenses in table wf with newer info.
wfPostgre.update(epdk_list_l0)
wfPostgre.update(epdk_list_l1)


version_no = input("Enter the new version number (current="+str(current_version)+"): ")
wfPostgre['version'] = version_no # TODO: Give version automatically on each run.

# reset the indexes (code will not run a 2. time if this is not done)
wfPostgre.reset_index(inplace=True)
epdk_list_l0.reset_index(inplace=True)
epdk_list_l1.reset_index(inplace=True)

# Add id numbers automatically
def add_incrementing_id(a, id_column, new_column_name):
    df = a
    #Filter the DataFrame to only include rows where id_column is not NaN
    df = df[df[id_column].notna()]
    df = df[df[id_column] < 10000]
    # Find the maximum id
    max_id = df[id_column].apply(lambda x: (int(float((str(x)[:4]))))).max()
    
    df = a
    df_na = df[df[id_column].isna()]
    df_na[id_column] = range(max_id + 1, max_id + 1 + len(df_na))
    df = pd.concat([df, df_na], axis=0, ignore_index=True, sort=False)
    df = df.dropna(subset=[id_column])
    return df

wfPostgre = add_incrementing_id(wfPostgre, 'id', 'id')



wfPostgre.to_sql('wf', ssh_db.engine, if_exists='append', index=False)

# Stop the SSH tunnel
ssh_db.stop_ssh_db()

wfPostgre